In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from datetime import date

In [2]:
def get_page(url):
    response = get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

base_ff_url = 'https://www.fleaflicker.com/mlb/leagues/'
league_ids = ['21579', '21581', '21580', '21582', '21583', '21584', '21585', '21586', '21587', '21588', '21589', 
              '21590', '21591', '21592', '21593', '21594', '21595', '21596']

all_teams = []
for l in league_ids:
    url = base_ff_url + l
    soup = get_page(url)
    trs = soup.find_all('tr')
    raw_headers = trs[1].find_all('th')
    player_data = trs[2:]
    headers = []
    for header in raw_headers:
        if header.text:
            headers.append(header.text)
    exp_headers = headers + ['league_id', 'league_name', 'team_id']   
    league_name = soup.find_all('li', {'class': 'active'})[1].text.strip()
    for row in player_data:
        d_dict = dict.fromkeys(exp_headers)
        d_dict['league_id'] = l
        d_dict['league_name'] = league_name
        d_dict['Team'] = row.find('td', {'class': 'left'}).text
        d_dict['Owner'] = row.find('td', {'class': 'right'}).text
        d_dict['team_id'] = row.find('a', href=True).get('href')[-6:]
        try:
            d_dict['Rank'] = row.find_all('td', {'class': 'right text-center'})[-1].text
        except IndexError:
            d_dict['Rank'] = row.find_all('td', {'class': 'bottom right text-center'})[-1].text
        heads = ['HR', 'R','RBI','SB','OBP','OPS','SO','SV','HD','ERA','WHP','QS']
        if d_dict['Owner'] == 'Take Over':
            stats = row.find_all('span', {'class': 'nowrap'})
        else:
            stats = row.find_all('span', {'class': 'nowrap'})[1:]
        for h, s in zip(heads, stats):
            d_dict[h] = s.text
        all_teams.append(d_dict)

In [3]:
all_df = pd.DataFrame(all_teams, columns=exp_headers)
all_df.HR = all_df.HR.astype(int)
all_df.R = all_df.R.astype(int)
all_df.RBI = all_df.RBI.astype(int)
all_df.SB = all_df.SB.astype(int)
all_df.OBP = all_df.OBP.astype(float)
all_df.OPS = all_df.OPS.astype(float)
all_df.SO = all_df.SO.astype(int)
all_df.SV = all_df.SV.astype(int)
all_df.HD = all_df.HD.astype(int)
all_df.ERA = all_df.ERA.astype(float)
all_df.WHP = all_df.WHP.astype(float)
all_df.QS = all_df.QS.astype(int)

In [4]:
rank_headers = ['HR', 'R','RBI','SB','OBP','OPS','SO','SV','HD','ERA','WHP','QS']
for r in rank_headers:
    if r in ['ERA', 'WHP']:
        all_df[r+'_Points'] = all_df[r].rank(ascending=False)
    else:
        all_df[r+'_Points'] = all_df[r].rank()

In [5]:
all_df['Total_Points'] = all_df.iloc[:,-12:].sum(axis=1)
all_df['Overall_Rank'] = all_df.Total_Points.rank(ascending=False)

In [6]:
all_df.head()

,Team,Owner,HR,R,RBI,SB,OBP,OPS,SO,SV,...,OBP_Points,OPS_Points,SO_Points,SV_Points,HD_Points,ERA_Points,WHP_Points,QS_Points,Total_Points,Overall_Rank
0,Bang Bang Pop Pop,JohnnyFang,126,431,377,40,0.343,0.820,772,44,...,114.0,121.0,200.0,163.5,212.0,123.5,150.5,148.0,1834.0,30.0
1,Forearm Tightness,CropRhombus,129,419,393,47,0.342,0.840,698,10,...,106.5,177.5,136.0,12.0,200.5,135.5,198.5,138.5,1770.0,44.0
2,Swinger Bell,WMason,128,422,399,36,0.355,0.840,733,23,...,199.5,177.5,172.5,55.5,196.5,75.5,111.0,129.0,1726.5,52.0
3,Harper Wallbanger,Prime416,149,420,445,39,0.336,0.824,746,19,...,57.5,137.5,185.0,41.0,164.5,82.0,84.0,60.0,1552.0,85.0
4,Ten Points for Griffeyndor,coreyjro,114,421,361,26,0.343,0.806,736,34,...,114.0,82.5,175.5,110.0,219.5,143.5,84.0,129.0,1439.5,101.0


In [7]:
t_date = str(date.today())
all_df.to_csv('current_rankings_'+t_date+'.csv')